In [1]:
import sys
import h2o
import pandas as pd
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /home/borja/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpam7dxh4t
  JVM stdout: /tmp/tmpam7dxh4t/h2o_borja_started_from_python.out
  JVM stderr: /tmp/tmpam7dxh4t/h2o_borja_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.31.0.5213
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_borja_2nd914
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.881 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
df = h2o.import_file(path="output/train_all.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
df.shape

(40455, 14)

In [5]:
display(df.head())

id,carat,cut,color,clarity,depth,table,x,y,z,price,color2,cut2,clarity2
0,1.12,Premium,H,SI1,61.6,59,6.67,6.63,4.1,5363,4,3,2
1,1.14,Very Good,E,SI2,60,54,6.74,6.97,4.11,5593,1,4,3
2,0.9,Very Good,D,SI2,60.3,63,6.12,6.22,3.72,3534,0,4,3
3,0.71,Ideal,E,VS1,61.9,54,5.74,5.76,3.56,3212,1,2,4
4,0.34,Very Good,F,SI2,60,62,4.51,4.55,2.72,447,2,4,3
5,0.33,Very Good,E,VS2,63.1,57,4.38,4.43,2.78,672,1,4,5
6,0.55,Premium,I,VVS1,61.3,59,5.27,5.3,3.24,1593,5,3,6
7,1.19,Premium,D,SI2,58.8,58,7.04,6.9,4.1,5504,0,3,3
8,0.41,Premium,E,VS1,60.4,58,4.78,4.82,2.9,1079,1,3,4
9,0.3,Premium,G,SI2,62.5,59,4.27,4.31,2.68,394,3,3,3


In [6]:
test = h2o.import_file(path="inputs/predict.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
test = test.as_data_frame()

In [8]:

test = test[["carat","cut","color","clarity","x","y","z"]]
test.head()

,carat,cut,color,clarity,x,y,z
0,0.33,Very Good,I,IF,4.44,4.46,2.76
1,1.21,Very Good,D,SI2,6.77,6.83,4.24
2,1.06,Very Good,D,SI1,6.64,6.71,3.96
3,0.36,Ideal,E,VVS1,4.64,4.61,2.54
4,0.70,Ideal,E,VS1,5.67,5.72,3.55


In [9]:
def cutchanger(x):
    if x== "Ideal":
        return 2
    elif x== "Premium":
        return 3
    elif x == "Very Good":
        return 4
    elif x== "Good":
        return 1
    else:
        return 0
    
def colorchanger(x):
    if x== "G":
        return 3
    elif x== "E":
        return 1
    elif x == "F":
        return 2
    elif x== "H":
        return 4
    elif x== "D":
        return 0
    elif x== "I":
        return 5
    else:
        return 6
    
def clarytychanger(x):
    if x== "SI1":
        return 2
    elif x== "VS2":
        return 5
    elif x == "SI2":
        return 3
    elif x== "VS1":
        return 4
    elif x== "VVS2":
        return 7
    elif x== "VVS1":
        return 6
    elif x== "IF":
        return 1
    else:
        return 0

In [10]:
test["cut2"] = test["cut"].apply(cutchanger)
test["color2"] = test["color"].apply(colorchanger)
test["clarity2"] = test["clarity"].apply(clarytychanger)

In [11]:
test.head()

,carat,cut,color,clarity,x,y,z,cut2,color2,clarity2
0,0.33,Very Good,I,IF,4.44,4.46,2.76,4,5,1
1,1.21,Very Good,D,SI2,6.77,6.83,4.24,4,0,3
2,1.06,Very Good,D,SI1,6.64,6.71,3.96,4,0,2
3,0.36,Ideal,E,VVS1,4.64,4.61,2.54,2,1,6
4,0.70,Ideal,E,VS1,5.67,5.72,3.55,2,1,4


In [12]:
response= "price"

In [13]:
predictors = ["carat","color2","cut2","clarity2","x","y","z"]

In [12]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model2 = H2OAutoML(verbosity = "warn")
#for i in range(0,5):
train, valid = df.split_frame(ratios = [.8])
diamons_model2.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
17:09:29.440: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [13]:
diamons_model2.leaderboard


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201005_170929,296328,544.36,296328,272.154,0.0934735
StackedEnsemble_BestOfFamily_AutoML_20201005_170929,301766,549.333,301766,277.232,nan
GBM_4_AutoML_20201005_170929,307080,554.148,307080,279.251,nan
GBM_grid__1_AutoML_20201005_170929_model_34,307738,554.741,307738,279.088,0.0962785
GBM_grid__1_AutoML_20201005_170929_model_30,308822,555.717,308822,283.673,0.100871
GBM_grid__1_AutoML_20201005_170929_model_13,312278,558.818,312278,291.125,0.107641
GBM_2_AutoML_20201005_170929,312712,559.207,312712,288.6,nan
GBM_grid__1_AutoML_20201005_170929_model_26,313459,559.874,313459,282.484,0.0985353
GBM_3_AutoML_20201005_170929,314248,560.578,314248,287.008,nan
GBM_grid__1_AutoML_20201005_170929_model_12,316508,562.59,316508,279.453,nan


In [196]:
display(test)
test = h2o.H2OFrame(test[["carat","color2","cut2","clarity2","x","y","z"]].reset_index())
pred = diamons_model2.leader.predict(test)
pred

,carat,cut,color,clarity,x,y,z,cut2,color2,clarity2
0,0.33,Very Good,I,IF,4.44,4.46,2.76,4,5,1
1,1.21,Very Good,D,SI2,6.77,6.83,4.24,4,0,3
2,1.06,Very Good,D,SI1,6.64,6.71,3.96,4,0,2
3,0.36,Ideal,E,VVS1,4.64,4.61,2.54,2,1,6
4,0.70,Ideal,E,VS1,5.67,5.72,3.55,2,1,4
...,...,...,...,...,...,...,...,...,...,...
13480,2.04,Good,D,SI1,8.15,8.11,5.03,1,0,2
13481,0.31,Ideal,G,IF,4.37,4.39,2.70,2,3,1
13482,0.52,Ideal,D,VVS2,5.14,5.18,3.20,2,0,7
13483,0.50,Premium,F,SI1,5.23,5.19,3.08,3,2,2


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
741.071
6020.29
5860.77
1094.31
3242.59
527.861
1451.2
860.545
8478.68
623.155


In [197]:
test["price"] = pred
test.head()

index,carat,color2,cut2,clarity2,x,y,z,price
0,0.33,5,4,1,4.44,4.46,2.76,741.071
1,1.21,0,4,3,6.77,6.83,4.24,6020.29
2,1.06,0,4,2,6.64,6.71,3.96,5860.77
3,0.36,1,2,6,4.64,4.61,2.54,1094.31
4,0.7,1,2,4,5.67,5.72,3.55,3242.59
5,0.3,5,3,7,4.28,4.37,2.67,527.861
6,0.52,1,3,2,5.3,5.21,3.1,1451.2
7,0.32,3,2,6,4.39,4.4,2.72,860.545
8,1.51,2,1,3,7.22,7.28,4.56,8478.68
9,0.42,6,2,5,4.79,4.86,2.96,623.155


In [198]:
upload = test[["index","price"]]

In [199]:
upload.rename(columns={'index': 'id'})
upload

id,price
0,741.071
1,6020.29
2,5860.77
3,1094.31
4,3242.59
5,527.861
6,1451.2
7,860.545
8,8478.68
9,623.155


In [200]:
upload = upload.as_data_frame()

In [201]:


upload

,id,price
0,0,741.070832
1,1,6020.287084
2,2,5860.766629
3,3,1094.308954
4,4,3242.594916
...,...,...
13480,13480,16641.203582
13481,13481,901.644896
13482,13482,2520.101971
13483,13483,1287.148178


In [202]:
upload.to_csv(r'output/pred4.csv',index=False,header=True)

In [14]:
df2 = h2o.import_file(path="inputs/train.csv")
test2 = h2o.import_file(path="inputs/predict.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
response= "price"

In [16]:
predictors = ["carat","color","cut","clarity","x","y","z"]

In [14]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model3 = H2OAutoML(verbosity = "warn")
#for i in range(0,5):
train, valid = df2.split_frame(ratios = [.8])
diamons_model3.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
20:50:10.546: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [15]:
diamons_model3.leaderboard


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201005_205010,277757,527.027,277757,266.871,0.0932635
StackedEnsemble_BestOfFamily_AutoML_20201005_205010,283206,532.171,283206,270.866,0.0953519
GBM_grid__1_AutoML_20201005_205010_model_30,291400,539.815,291400,270.993,0.0924741
GBM_grid__1_AutoML_20201005_205010_model_37,291973,540.345,291973,278.475,0.102636
GBM_grid__1_AutoML_20201005_205010_model_88,292603,540.928,292603,271.606,0.0940889
GBM_1_AutoML_20201005_205010,292873,541.178,292873,281.222,0.103142
GBM_3_AutoML_20201005_205010,292969,541.266,292969,275.851,0.0991401
GBM_grid__1_AutoML_20201005_205010_model_77,293118,541.403,293118,284.877,0.106464
GBM_grid__1_AutoML_20201005_205010_model_22,293353,541.621,293353,273.787,0.0973496
GBM_grid__1_AutoML_20201005_205010_model_93,294063,542.275,294063,271.254,0.0908826


In [17]:
predictors2 = ["carat","color","cut","clarity","depth","table","x","y","z"]

In [18]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model4 = H2OAutoML(verbosity = "warn")
#for i in range(0,5):
train, valid = df2.split_frame(ratios = [.8])
diamons_model4.train(x=predictors2,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
07:23:10.697: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [19]:
diamons_model4.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_072310,285690,534.5,285690,264.931,0.0911306
StackedEnsemble_BestOfFamily_AutoML_20201006_072310,288417,537.045,288417,269.177,0.0937828
GBM_grid__1_AutoML_20201006_072310_model_15,296287,544.323,296287,275.559,0.0963073
GBM_2_AutoML_20201006_072310,296362,544.391,296362,280.089,0.103858
GBM_1_AutoML_20201006_072310,296507,544.525,296507,280.561,0.101452
GBM_grid__1_AutoML_20201006_072310_model_38,297091,545.06,297091,273.355,0.0904589
GBM_4_AutoML_20201006_072310,297144,545.109,297144,272.7,0.0923862
GBM_grid__1_AutoML_20201006_072310_model_81,297257,545.213,297257,271.312,0.0901792
GBM_3_AutoML_20201006_072310,297428,545.369,297428,276.751,0.0982561
GBM_grid__1_AutoML_20201006_072310_model_73,298282,546.152,298282,271.828,0.0895159


In [20]:
df3 = h2o.import_file(path="output/train_all.csv")
test3 = h2o.import_file(path="inputs/predict.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
test3 = test3.as_data_frame()

In [22]:
test3["cut"] = test3["cut"].apply(cutchanger)
test3["color"] = test3["color"].apply(colorchanger)
test3["clarity"] = test3["clarity"].apply(clarytychanger)

In [23]:
response= "price"

In [24]:
predictors = ["carat","color","cut","clarity","table","depth","x","y","z"]

In [ ]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model5 = H2OAutoML(verbosity = "warn")
#for i in range(0,5):
train, valid = df.split_frame(ratios = [.8])
diamons_model5.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
08:18:50.954: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████

In [ ]:
diamons_model5.leaderboard